In [1]:
import pickle

def load_embeddings_and_metadata(filename="data/pickle/embeddings_metadata.pkl"):
    # Laden der Embeddings und Metadaten aus der Pickle-Datei
    with open(filename, "rb") as file:
        data = pickle.load(file)
    
    embeddings = data['embeddings']
    metadatas = data['metadatas']
    
    print(f"Die Embeddings und Metadaten wurden erfolgreich aus '{filename}' geladen.")
    return embeddings, metadatas

# Beispielnutzung:
embeddings, metadatas = load_embeddings_and_metadata(filename="data/pickle/embeddings_metadata.pkl")

Die Embeddings und Metadaten wurden erfolgreich aus 'data/pickle/embeddings_metadata.pkl' geladen.


In [2]:
# Anzahl der Einträge, die Du anzeigen möchtest
num_examples = min(5, len(embeddings))  # Setzt num_examples auf 5 oder weniger, wenn nicht genug Embeddings vorhanden sind

# Ausgabe der ersten paar Embeddings und ihrer zugehörigen Metadaten
for i in range(num_examples):
    print(f"Chunk {i}:\n")
    print(f"Embedding: {embeddings[i][:10]}...")  # Zeigt nur die ersten 10 Werte des Embeddings
    print(f"Metadaten: {metadatas[i]}")
    print("\n" + "="*50 + "\n")

Chunk 0:

Embedding: [ 0.19252267 -0.5782604  -0.19584805  0.02244737  0.38322002 -0.13617823
  0.5281439  -0.08208066 -0.31730434  0.15474701]...
Metadaten: {'text': 'batterien akkumulatoren gehören nicht brände auslösen umweltgefährdende stoffe enthalten mensch umwelt belasten ressourcen rohstoffe sammelboxen handel batterien verkauft supermärkte drogeriemärkte warenhäuser baumärkte tankstellen einpacktische kommunale sammelstellen wertstoffhöfe schadstoffmobile freiwillige sammelstellen unternehmen behörden hochschulen nicht batterien elektrogeräten entsorgung entnehmen batteriesammlung geben', 'keywords': ['wertstoffhof']}


Chunk 1:

Embedding: [ 0.17300805 -0.2980221  -0.29367456  0.17560288  0.06461029 -0.10351696
  0.5241277   0.19690768 -0.01365028  0.18295565]...
Metadaten: {'text': 'gelbe tonne gelbe tonne gelber sack leeren verpackungen außer glas papier kunststoff metallen aluminium getränkebecher tipps verpackungen materialien trennen nicht stopfen recycelt deckel joghurt

In [ ]:
def display_embeddings_and_metadata(embeddings, metadatas, num_examples=5):
    # Anzahl der Einträge, die Du anzeigen möchtest, begrenzen
    num_examples = min(num_examples, len(embeddings))  # Setzt num_examples auf 5 oder weniger, wenn nicht genug Embeddings vorhanden sind
    
    # Ausgabe der Embeddings und ihrer zugehörigen Metadaten
    for i in range(num_examples):
        print(f"Chunk {i}:\n")
        print(f"Embedding: {embeddings[i][:10]}...")  # Zeigt nur die ersten 10 Werte des Embeddings
        print(f"Metadaten: {metadatas[i]}")
        print("\n" + "="*50 + "\n")

# Beispielnutzung:
display_embeddings_and_metadata(embeddings, metadatas, num_examples=5)

In [ ]:
import chromadb

def delete_chroma_collection(collection_name='document_embeddings_v2'):
    # Erstelle oder öffne eine Chroma-Datenbank
    client = chromadb.Client()

    # Prüfe, ob die Collection existiert
    if collection_name in [c.name for c in client.list_collections()]:
        # Lösche die Collection
        client.delete_collection(collection_name)
        print(f"Collection '{collection_name}' erfolgreich gelöscht.")
    else:
        print(f"Collection '{collection_name}' existiert nicht.")

# Beispielnutzung:
delete_chroma_collection(collection_name='document_embeddings_v2')

In [ ]:
import chromadb

def create_chroma_database_v2(embeddings, metadatas, collection_name='document_embeddings_v2'):
    # Erstelle oder öffne eine Chroma-Datenbank
    client = chromadb.Client()
    
    # Erstelle eine neue Collection in der Datenbank mit einem anderen Namen
    collection = client.create_collection(name=collection_name)
    
    # Konvertiere Embeddings von numpy array in Liste von Listen
    embeddings_as_list = [embedding.tolist() for embedding in embeddings]
    
    # Füge die Embeddings und Metadaten zur Collection hinzu
    for i, (embedding, metadata) in enumerate(zip(embeddings_as_list, metadatas)):
        # Metadaten umwandeln, sodass die Liste der Keywords als String gespeichert wird
        metadata['keywords'] = ','.join(metadata['keywords'])
        
        collection.add(
            embeddings=[embedding],
            metadatas=[metadata],
            ids=[f"doc_{i}"]
        )
    
    print(f"Chroma-Datenbank '{collection_name}' erfolgreich erstellt und {len(embeddings)} Embeddings hinzugefügt.")

# Beispielnutzung:
create_chroma_database_v2(embeddings, metadatas)

In [ ]:
import chromadb
from sentence_transformers import SentenceTransformer

def query_chroma_database(query_text, collection_name='document_embeddings_v2', top_k=5, model_name='sentence-transformers/multi-qa-mpnet-base-dot-v1'):
    # Lade das vortrainierte Modell
    model = SentenceTransformer(model_name)

    # Erstelle oder öffne eine Chroma-Datenbank
    client = chromadb.Client()

    # Öffne die Collection
    collection = client.get_collection(name=collection_name)

    # Erstelle ein Embedding für die Abfrage
    query_embedding = model.encode(query_text).tolist()

    # Suche in der Collection nach den besten Übereinstimmungen
    results = collection.query(query_embeddings=[query_embedding], n_results=top_k)

    return results

# Beispielnutzung:
query_text = "Wie entsorge ich Altglas in Frankfurt?"
results = query_chroma_database(query_text)

# Ausgabe der Ergebnisse
for i, (doc_id, metadata) in enumerate(zip(results['ids'], results['metadatas'])):
    print(f"Ergebnis {i+1}:")
    print(f"ID: {doc_id}")
    print(f"Metadaten: {metadata}")
    print("="*50)

In [ ]:
def hybrid_search_v2(chroma_client, query, keywords, embedding_model, collection_name="document_embeddings_v2", top_k=10):
    # 1. Semantische Suche: Finde ähnliche Dokumente basierend auf dem Query-Embedding
    query_embedding = embedding_model.encode(query).tolist()
    collection = chroma_client.get_collection(name=collection_name)
    semantic_results = collection.query(
        query_embeddings=[query_embedding],
        n_results=top_k
    )
    
    # 2. Klassische Textsuche: Finde exakte Übereinstimmungen für Keywords
    text_search_results = []
    for i in range(len(semantic_results['ids'][0])):
        metadata = semantic_results['metadatas'][0][i]
        if isinstance(metadata, dict) and any(keyword.lower() in metadata['text'].lower() for keyword in keywords):
            text_search_results.append({
                'id': semantic_results['ids'][0][i],
                'text': metadata['text'],
                'score': 1.0  # Hohe Gewichtung für exakte Übereinstimmung
            })
    
    # 3. Kombination der Ergebnisse: Füge Gewichtung basierend auf der Suchmethode hinzu
    combined_results = []
    for i in range(len(semantic_results['ids'][0])):
        # Hole den einzelnen Distanzwert
        distance = float(semantic_results['distances'][0][i])
        combined_score = 0.6 * (1 - distance)  # Angepasste Gewichtung für semantische Suche
        combined_results.append({
            'id': semantic_results['ids'][0][i],
            'text': semantic_results['metadatas'][0][i]['text'] if isinstance(semantic_results['metadatas'][0][i], dict) else "",
            'score': combined_score
        })
    
    combined_results.extend(text_search_results)
    
    # 4. Sortieren der kombinierten Ergebnisse nach Score (höchste zuerst)
    combined_results.sort(key=lambda x: x['score'], reverse=True)
    
    # 5. Ausgabe der Top-k Ergebnisse
    return combined_results[:top_k]

# Beispielnutzung:
query = "Wie entsorge ich Altglas in Frankfurt?"
keywords = ["altglascontainer", "restmüll", "wertstoffhof"]
hybrid_results_v2 = hybrid_search_v2(chroma_client, query, keywords, embedding_model)

# Ausgabe der hybriden Suchergebnisse
for i, result in enumerate(hybrid_results_v2):
    print(f"Ergebnis {i+1}:\n")
    print(f"ID: {result['id']}")
    print(f"Text: {result['text']}")
    print(f"Score: {result['score']}")
    print("\n" + "="*50 + "\n")

In [ ]:
import spacy

# Lade das deutsche Sprachmodell von spaCy
nlp = spacy.load("de_core_news_sm")

def extract_keywords_from_query(query, custom_stopwords=None):
    """
    Extrahiert Keywords aus einer Query durch Tokenisierung und Entfernung von Stoppwörtern.
    """
    if custom_stopwords is None:
        custom_stopwords = set(nlp.Defaults.stop_words)
    
    # Tokenisiere die Query
    doc = nlp(query.lower())
    
    # Extrahiere Keywords: Wörter, die keine Stoppwörter, Zahlen oder Satzzeichen sind
    keywords = [token.text for token in doc if token.text not in custom_stopwords and not token.is_punct and not token.like_num]
    
    return keywords

# Beispielnutzung:
query2 = "Wie entsorge ich Altglas in Frankfurt?"
keywords2 = extract_keywords_from_query(query2)

# test chroma

#hybrid_results_v21 = hybrid_search_v2(chroma_client, query2, keywords2, embedding_model)

#print(f"Query: {query2}")
#print(f"Extrahierte Keywords: {keywords2}")
# Ausgabe der hybriden Suchergebnisse
#for i, result in enumerate(hybrid_results_v21):
#    print(f"Ergebnis {i+1}:\n")
 #   print(f"ID: {result['id']}")
  #  print(f"Text: {result['text']}")
   # print(f"Score: {result['score']}")
    #print("\n" + "="*50 + "\n")

In [ ]:
# Deduplizierung und kombinierte Gewichtung verbessern
def hybrid_search_v3(chroma_client, query, keywords, embedding_model, collection_name="document_embeddings_v2", top_k=10):
    # 1. Semantische Suche
    query_embedding = embedding_model.encode(query).tolist()
    collection = chroma_client.get_collection(name=collection_name)
    semantic_results = collection.query(
        query_embeddings=[query_embedding],
        n_results=top_k
    )
    
    # 2. Klassische Textsuche
    text_search_results = []
    for i in range(len(semantic_results['ids'][0])):
        metadata = semantic_results['metadatas'][0][i]
        if isinstance(metadata, dict) and any(keyword.lower() in metadata['text'].lower() for keyword in keywords):
            text_search_results.append({
                'id': semantic_results['ids'][0][i],
                'text': metadata['text'],
                'keywords': keywords,
                'score': 1.0  # Hohe Gewichtung für exakte Übereinstimmung
            })
    
    # 3. Kombination der Ergebnisse
    combined_results = []
    seen_ids = set()  # Set zur Deduplizierung
    for i in range(len(semantic_results['ids'][0])):
        distance = float(semantic_results['distances'][0][i])
        combined_score = 0.6 * (1 - distance)  # Angepasste Gewichtung für semantische Suche
        
        if semantic_results['ids'][0][i] not in seen_ids:
            seen_ids.add(semantic_results['ids'][0][i])
            combined_results.append({
                'id': semantic_results['ids'][0][i],
                'text': semantic_results['metadatas'][0][i]['text'] if isinstance(semantic_results['metadatas'][0][i], dict) else "",
                'keywords': keywords,
                'score': combined_score
            })
    
    for result in text_search_results:
        if result['id'] not in seen_ids:
            combined_results.append(result)
    
    # 4. Sortieren der kombinierten Ergebnisse nach Score
    combined_results.sort(key=lambda x: x['score'], reverse=True)
    
    # 5. Normierung der Scores (falls nötig)
    min_score = min([r['score'] for r in combined_results])
    if min_score < 0:
        for result in combined_results:
            result['score'] += abs(min_score)
    
    # 6. Ausgabe der Top-k Ergebnisse
    return combined_results[:top_k]

# Beispielnutzung:
query4 = "Wie entsorge ich Altglas in Frankfurt?"
keywords4 = extract_keywords_from_query(query4)

hybrid_results_v3 = hybrid_search_v3(chroma_client, query4, keywords4, embedding_model)

print(f"Query: {query4}")
print(f"Extrahierte Keywords: {keywords4}")
# Ausgabe der hybriden Suchergebnisse
for i, result in enumerate(hybrid_results_v3):
    print(f"Ergebnis {i+1}:\n")
    print(f"ID: {result['id']}")
    print(f"Text: {result['text']}")
    print(f"Keywords: {result.get('keywords', [])}")
    print(f"Score: {result['score']}")
    print("\n" + "="*50 + "\n")

In [ ]:
# Erweiterte Version der hybriden Suche mit normalisierter Distanzberechnung, Version 5
def hybrid_search_v5(chroma_client, query, keywords, embedding_model, collection_name="document_embeddings_v2", top_k=10):
    # 1. Semantische Suche
    query_embedding = embedding_model.encode(query).tolist()
    collection = chroma_client.get_collection(name=collection_name)
    semantic_results = collection.query(
        query_embeddings=[query_embedding],
        n_results=top_k
    )
    
    # 2. Distanznormalisierung
    distances = semantic_results['distances'][0]
    min_distance = min(distances)
    max_distance = max(distances)
    normalized_distances = [(d - min_distance) / (max_distance - min_distance) for d in distances]
    
    # 3. Klassische Textsuche basierend auf Keywords
    text_search_results = []
    for i in range(len(semantic_results['ids'][0])):
        metadata = semantic_results['metadatas'][0][i]
        keyword_count = sum(keyword.lower() in metadata['text'].lower() for keyword in keywords)
        if keyword_count > 0:
            text_search_results.append({
                'id': semantic_results['ids'][0][i],
                'text': metadata['text'],
                'keywords': keywords,
                'score': 1.0 + 0.5 * keyword_count  # Gewichtung für Keywords
            })
    
    # 4. Kombination der Ergebnisse
    combined_results = []
    seen_ids = set()
    for i in range(len(semantic_results['ids'][0])):
        distance = normalized_distances[i]
        combined_score = 1.0 - distance  # Gewichtung für semantische Suche
        
        if semantic_results['ids'][0][i] not in seen_ids:
            seen_ids.add(semantic_results['ids'][0][i])
            combined_results.append({
                'id': semantic_results['ids'][0][i],
                'text': semantic_results['metadatas'][0][i]['text'] if isinstance(semantic_results['metadatas'][0][i], dict) else "",
                'keywords': keywords,
                'score': combined_score
            })
    
    for result in text_search_results:
        if result['id'] not in seen_ids:
            combined_results.append(result)
    
    # 5. Sortieren der kombinierten Ergebnisse nach Score
    combined_results.sort(key=lambda x: x['score'], reverse=True)
    
    # 6. Ausgabe der Top-k Ergebnisse
    return combined_results[:top_k]

# Beispielnutzung:
query5 = "Wie entsorge ich Altglas in Frankfurt?"
keywords5 = extract_keywords_from_query(query5)

# Durchführung der hybriden Suche mit Keywords
hybrid_results_v5 = hybrid_search_v5(chroma_client, query5, keywords5, embedding_model)

print(f"Query: {query5}")
print(f"Extrahierte Keywords: {keywords5}")
# Ausgabe der hybriden Suchergebnisse
for i, result in enumerate(hybrid_results_v5):
    print(f"Ergebnis {i+1}:\n")
    print(f"ID: {result['id']}")
    print(f"Text: {result['text']}")
    print(f"Keywords: {result.get('keywords', [])}")
    print(f"Score: {result['score']}")
    print("\n" + "="*50 + "\n")

In [ ]:
from sentence_transformers import SentenceTransformer
import chromadb

# Lade das vortrainierte Modell
model_name = 'sentence-transformers/multi-qa-mpnet-base-dot-v1'
embedding_model = SentenceTransformer(model_name)

# Initialisiere den Chroma-Client
chroma_client = chromadb.Client()

# Überprüfe, ob die Collection existiert
def get_existing_collection(chroma_client, collection_name="document_embeddings_v2"):
    try:
        collection = chroma_client.get_collection(name=collection_name)
        print(f"Collection {collection_name} erfolgreich geladen.")
        return collection
    except ValueError:
        print(f"Collection {collection_name} existiert nicht.")
        return None

# Zugriff auf die bestehende Collection
collection = get_existing_collection(chroma_client, "document_embeddings_v2")

if collection is None:
    print("Die angegebene Collection existiert nicht. Bitte überprüfe den Collection-Namen.")
else:
    # Erweiterte Version der hybriden Suche mit normalisierter Distanzberechnung, Version 5
    def hybrid_search_v5(chroma_client, query, keywords, embedding_model, collection_name="document_embeddings_v2", top_k=10):
        # 1. Semantische Suche
        query_embedding = embedding_model.encode(query).tolist()
        collection = chroma_client.get_collection(name=collection_name)
        semantic_results = collection.query(
            query_embeddings=[query_embedding],
            n_results=top_k
        )
        
        # 2. Distanznormalisierung
        distances = semantic_results['distances'][0]
        min_distance = min(distances)
        max_distance = max(distances)
        normalized_distances = [(d - min_distance) / (max_distance - min_distance) for d in distances]
        
        # 3. Klassische Textsuche basierend auf Keywords
        text_search_results = []
        for i in range(len(semantic_results['ids'][0])):
            metadata = semantic_results['metadatas'][0][i]
            keyword_count = sum(keyword.lower() in metadata['text'].lower() for keyword in keywords)
            if keyword_count > 0:
                text_search_results.append({
                    'id': semantic_results['ids'][0][i],
                    'text': metadata['text'],
                    'keywords': keywords,
                    'score': 1.0 + 0.5 * keyword_count  # Gewichtung für Keywords
                })
        
        # 4. Kombination der Ergebnisse
        combined_results = []
        seen_ids = set()
        for i in range(len(semantic_results['ids'][0])):
            distance = normalized_distances[i]
            combined_score = 1.0 - distance  # Gewichtung für semantische Suche
            
            if semantic_results['ids'][0][i] not in seen_ids:
                seen_ids.add(semantic_results['ids'][0][i])
                combined_results.append({
                    'id': semantic_results['ids'][0][i],
                    'text': semantic_results['metadatas'][0][i]['text'] if isinstance(semantic_results['metadatas'][0][i], dict) else "",
                    'keywords': keywords,
                    'score': combined_score
                })
        
        for result in text_search_results:
            if result['id'] not in seen_ids:
                combined_results.append(result)
        
        # 5. Sortieren der kombinierten Ergebnisse nach Score
        combined_results.sort(key=lambda x: x['score'], reverse=True)
        
        # 6. Ausgabe der Top-k Ergebnisse
        return combined_results[:top_k]

    # Beispielnutzung:
    query5 = "Wie entsorge ich Altglas in Frankfurt?"
    keywords5 = extract_keywords_from_query(query5)

    # Durchführung der hybriden Suche mit Keywords
    hybrid_results_v5 = hybrid_search_v5(chroma_client, query5, keywords5, embedding_model)

    print(f"Query: {query5}")
    print(f"Extrahierte Keywords: {keywords5}")
    # Ausgabe der hybriden Suchergebnisse
    for i, result in enumerate(hybrid_results_v5):
        print(f"Ergebnis {i+1}:\n")
        print(f"ID: {result['id']}")
        print(f"Text: {result['text']}")
        print(f"Keywords: {result.get('keywords', [])}")
        print(f"Score: {result['score']}")
        print("\n" + "="*50 + "\n")

In [4]:
import chromadb
from chromadb.config import DEFAULT_TENANT, DEFAULT_DATABASE, Settings

def create_chroma_database_v2(embeddings, metadatas, collection_name='document_embeddings_v2', persist_directory='chroma_db'):
    # Erstelle oder öffne eine Chroma-Datenbank mit Persistierung
    client = chromadb.PersistentClient(
        path=persist_directory,
        settings=Settings(),
        tenant=DEFAULT_TENANT,
        database=DEFAULT_DATABASE,
    )
    
    # Erstelle eine neue Collection in der Datenbank oder lade eine bestehende
    collection = client.get_or_create_collection(name=collection_name)
    
    # Konvertiere Embeddings von numpy array in Liste von Listen
    embeddings_as_list = [embedding.tolist() for embedding in embeddings]
    
    # Füge die Embeddings und Metadaten zur Collection hinzu
    for i, (embedding, metadata) in enumerate(zip(embeddings_as_list, metadatas)):
        # Metadaten umwandeln, sodass die Liste der Keywords als String gespeichert wird
        metadata['keywords'] = ','.join(metadata['keywords'])
        
        collection.add(
            embeddings=[embedding],
            metadatas=[metadata],
            ids=[f"doc_{i}"]
        )
    
    print(f"Chroma-Datenbank '{collection_name}' erfolgreich erstellt und {len(embeddings)} Embeddings hinzugefügt.")

# Beispielnutzung:
create_chroma_database_v2(embeddings, metadatas)

Chroma-Datenbank 'document_embeddings_v2' erfolgreich erstellt und 8 Embeddings hinzugefügt.


In [3]:
import chromadb
from chromadb.config import DEFAULT_TENANT, DEFAULT_DATABASE, Settings

def delete_chroma_collection(collection_name='document_embeddings_v2', persist_directory='chroma_db'):
    # Öffne die Chroma-Datenbank mit Persistierung
    client = chromadb.PersistentClient(
        path=persist_directory,
        settings=Settings(),
        tenant=DEFAULT_TENANT,
        database=DEFAULT_DATABASE,
    )
    
    # Überprüfe, ob die Collection existiert, und lösche sie, wenn ja
    existing_collections = [col.name for col in client.list_collections()]
    if collection_name in existing_collections:
        client.delete_collection(name=collection_name)
        print(f"Collection '{collection_name}' erfolgreich gelöscht.")
    else:
        print(f"Collection '{collection_name}' nicht gefunden.")
        
delete_chroma_collection(collection_name='document_embeddings_v2')

Collection 'document_embeddings_v2' erfolgreich gelöscht.
